In [1]:
%pylab inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(precision=6, linewidth=110)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py

import lal
import lalsimulation

In [3]:
sys.path.insert(0, '../../src')

import waveform as wave
import waveformset as ws
import trainingset as train
import taylorf2 as f2
import gaussianprocessregression as gpr
import designofexperiment as doe
import lalwaveform
import plotparams
import greedy
import empiricalinterpolation as eim
import surrogate
import diagnostics
import uncertaintysampling as us

import imp
imp.reload(wave)
imp.reload(ws)
imp.reload(train)
imp.reload(f2)
imp.reload(gpr)
imp.reload(doe)
imp.reload(lalwaveform)
imp.reload(greedy)
imp.reload(eim)
imp.reload(surrogate)
imp.reload(diagnostics)
imp.reload(us)

import constants
imp.reload(constants)
from constants import *

# Construct surrogate in way that can be directly converted to lalsuite code

In [4]:
def kernel(x1, x2, hyperparams):
    """Matern covariance function for n-dimensional data.
    
    Parameters
    ----------
    x1 : array with shape ndim
    x2 : array with shape ndim
    hyperparams : array with shape ndim+2 [sigma_f, ls0, ls1, ..., sigma_n]
        sigma_f : Approximately the range (ymax-ymin) of values that the data takes.
            sigma_f^2 called the signal variance.
        sigma_n : Noise term. The uncertainty in the y values of the data.
        lsi : Length scales for the variation in dimension i.
    
    Returns
    -------
    covariance : float
    """
    sigma_f = hyperparams[0]
    sigma_n = hyperparams[-1]
    ls = hyperparams[1:-1]
    ndim = len(ls)
    
    # Noise nugget for diagonal elements
    if np.array_equal(x1, x2):
        nugget = sigma_n**2
    else:
        nugget = 0.0
    #nugget = sigma_n**2
    
    # r**2
    rsq = np.sum(np.array([(x1[i]-x2[i])**2 / ls[i]**2 for i in range(ndim)]))
    r = np.sqrt(rsq)
    
    # nu = 5/2 Matern covariance
    matern = (1. + np.sqrt(5.)*r + 5.*r**2/3.) * np.exp(-np.sqrt(5.)*r)
    
    # Full covariance
    # You must include the nugget to agree with scikit-learn when the points x1, x2 are exactly the same
    return sigma_f**2 * matern + nugget

In [5]:
def gp_predict(xst, hyperparams, x_train, Kinv_dot_y):
    """Interpolate the function at the point xst using Gaussian process regression.
    
    Parameters
    ----------
    xst : array of shape ndim.
        Point x_* where you want to evaluate the function.
    hyperparams : array with shape ndim+2 [sigma_f, ls0, ls1, ..., sigma_n].
        Hyperparameters for the GPR kernel.
    x_train : array of shape (n_train, ndim).
        Training set points.
    Kinv_dot_y : array of shape n_train.
        The interpolating weights at each training set point.
    
    Returns
    -------
    yst : float
        Interpolated value at the point xst.
    """
    # Evaluate vector K_*
    Kst = np.array([kernel(xst, x, hyperparams) for x in x_train])

    # Evaluate y_*
    return np.dot(Kst, Kinv_dot_y)

In [6]:
def extract_data_from_scikit_learn(gp):
    """Extract the data in the scikit-learn GaussianProcessRegressor class 
    that you need for the lalsuite version.
    """
    # hyperparams = np.array([sigma_f, lq, ls1, ls2, llam1, llam2, sigma_n])
    hyperparams = gpr.get_hyperparameters(gp)
    
    # The training data
    x_train = gp.X_train_
    y_train = gp.y_train_
    
    # Evaluate K
    K = np.array([[kernel(x1, x2, hyperparams) for x2 in x_train] for x1 in x_train])
    
    # Evaluate K^{-1}
    Kinv = np.linalg.inv(K)
    
    # Evaluate (K^{-1})_{ij} y_j (array of length nparams).
    Kinv_dot_y = np.dot(Kinv, y_train)
    
    return hyperparams, x_train, Kinv_dot_y

# Load scikit-learn (python) version of surrogate

In [7]:
# nodes_filename = '../../data/TEOBv4_20hz/nodes_corners_lhd.hdf5'
# gp_amp_filename = '../../data/TEOBv4_20hz/gp_spline_amp_corners_lhd.hdf5'
# gp_phase_filename = '../../data/TEOBv4_20hz/gp_spline_phase_corners_lhd.hdf5'
# sur = surrogate.GPSplineSurrogate.load(nodes_filename, gp_amp_filename, gp_phase_filename, order=3, npoints=10000)

# nodes_filename = '../../data/TEOBv4_20hz/nodes_lhd_uncsamp.hdf5'
# gp_amp_filename = '../../data/TEOBv4_20hz/gp_spline_amp_lhd_uncsamp.hdf5'
# gp_phase_filename = '../../data/TEOBv4_20hz/gp_spline_phase_lhd_uncsamp.hdf5'
# sur = surrogate.GPSplineSurrogate.load(nodes_filename, gp_amp_filename, gp_phase_filename, order=3, npoints=10000)

nodes_filename = '../../data/TEOBv4_20hz/nodes_lhd_uncsamp_rand.hdf5'
gp_amp_filename = '../../data/TEOBv4_20hz/gp_spline_amp_lhd_uncsamp_rand.hdf5'
gp_phase_filename = '../../data/TEOBv4_20hz/gp_spline_phase_lhd_uncsamp_rand.hdf5'
sur = surrogate.GPSplineSurrogate.load(nodes_filename, gp_amp_filename, gp_phase_filename, order=3, npoints=10000)

In [8]:
# Is there an alpha = 1.0e-10 term that in the scikit-learn version?

#Random point:
x = np.array([0.8, 0.2, 0.1, 1000, 2000])

# Point exactly in training set:
#x = np.array([1.0, 0.5, 0.5, 0.0, 0.0])

for i in range(len(sur.damp_gp_list)):
    gp = sur.damp_gp_list[i]
    gp.alpha = 0.0
    #gp.alpha_ *= 0.0
    #gp = sur.dphase_gp_list[0]
    hyperparams, x_train, Kinv_dot_y = extract_data_from_scikit_learn(gp)

    a = gp.predict(np.atleast_2d(x))[0]
    b = gp_predict(x, hyperparams, x_train, Kinv_dot_y)

    sigma_n = hyperparams[-1]
    print sigma_n
    print a, b, np.abs(b/a-1.), np.abs(b-a)

1.39600526984e-08
4.17487390696e-05 4.05352113715e-05 0.0290674095838 1.21352769814e-06
1.55035513171e-08
3.67810267562e-05 3.54895069352e-05 0.0351137511608 1.29151982096e-06
1.82730542679e-08
2.4008448531e-05 2.42074042498e-05 0.00828690444117 1.98955718757e-07
2.16124100206e-08
5.33448905858e-06 6.37387147773e-06 0.194841981629 1.03938241915e-06
2.55287288235e-08
-2.13503939751e-05 -2.15834811566e-05 0.0109172309307 2.33087181487e-07
3.01173964908e-08
-5.83986928766e-05 -6.68643463122e-05 0.144963063703 8.46565343565e-06
3.57215692049e-08
-0.000109408364918 -0.000109635626462 0.0020771861819 2.27261543792e-07
4.2322391786e-08
-0.000178969262105 -0.000179065543033 0.00053797465849 9.62809276611e-08
5.02496463388e-08
-0.000270962557051 -0.000270918033489 0.000164316289146 4.45235618721e-08
5.97939296131e-08
-0.000392716507345 -0.000392671397579 0.000114865977454 4.51097654786e-08
7.13092644237e-08
-0.000552732363941 -0.000552418991637 0.000566951249113 3.13372304162e-07
8.52627682315e

# Generate hdf5 file for lalsuite version

In [18]:
def lalsuite_spline_surrogate_format(filename, sur):
    """Write data to an hdf5 file format that can be read by the 
    lalsuite version of the code.
    """
    f = h5py.File(filename, libver='latest')
    
    namp = len(sur.damp_gp_list)
    nphase = len(sur.dphase_gp_list)
    
    # Make sure 'Description' is capitalized for LAL
    f.attrs['Description'] = \
'''
********************************************************************************
Data for the SEOBNRv4T_surrogate waveform (aligned-spin BNS with tidal interactions).

See B. Lackey, M. Puerrer, A. Taracchini. arXiv:xxxx.xxxx.

Parameter ranges:
* 1.0/3.0 <= q <= 1.0
* -0.5 <= spin_1z <= 0.5
* -0.5 <= spin_2z <= 0.5
* 0.0 <= lambda_1 <= 5000.0
* 0.0 <= lambda_2 <= 5000.0

This surrogate was built using the SEOBv4T waveform.

The first spline node for dphase is not listed since it is the same as the first
node for damp, and dphase = 0 for the first node.

The hyperparameters for the Gaussian process regression associated with each 
basis function are listed in the order
[sigma_f, l_q, l_spin1z, l_spin2z, l_lambda1, l_lambda2, sigma_n]
where sigma_f is approximately the function range, sigma_n is the noise/tolerance, 
and l_i is the correlation length scale for the parameter i.
********************************************************************************
'''
    
    f.attrs['Creator'] = 'Ben Lackey, Michael Puerrer, Andrea Taracchini'
    f.attrs['Email'] = 'benjamin.lackey@ligo.org, michael.puerrer@ligo.org'
    f.attrs.create('version_major', 1, dtype='i4')
    f.attrs.create('version_minor', 0, dtype='i4')
    f.attrs.create('version_micro', 0, dtype='i4')
    
    # Bounds
    # To be read properly from LAL, make sure that all bounds are arrays of floats *not* integers.
    f['q_bounds'] = np.array([1.0/3.0, 1.0])
    f['chi1_bounds'] = np.array([-0.5, 0.5])
    f['chi2_bounds'] = np.array([-0.5, 0.5])
    f['lambda1_bounds'] = np.array([0.0, 5000.0])
    f['lambda2_bounds'] = np.array([0.0, 5000.0])

    # Nodes for splines.
    f['spline_nodes_amp'] = sur.mf_amp
    f['spline_nodes_phase'] = sur.mf_phase
    
    print f['spline_nodes_amp'][:].shape
    print f['spline_nodes_phase'][:].shape
    
    # Training set samples.
    # They are the same for all basis functions so pick amp_0
    gp = sur.damp_gp_list[0]
    x_train = gp.X_train_
    f['x_train'] = x_train
    
    print 'Writing amplitude bases...'
    hyp_amp = []
    kinv_dot_y_amp = []
    for i in range(namp):
        print i, 
        gp = sur.damp_gp_list[i]
        hyperparameters, x_train, kinv_dot_y = extract_data_from_scikit_learn(gp)
        hyp_amp.append(hyperparameters)
        kinv_dot_y_amp.append(kinv_dot_y)
    
    f['hyp_amp'] = np.array(hyp_amp)
    f['kinv_dot_y_amp'] = np.array(kinv_dot_y_amp)
    
    print f['hyp_amp'][:].shape
    print f['kinv_dot_y_amp'][:].shape
    
    
    print '\nWriting phase bases...'
    hyp_phase = []
    kinv_dot_y_phase = []
    for i in range(nphase):
        print i, 
        gp = sur.dphase_gp_list[i]
        hyperparameters, x_train, kinv_dot_y = extract_data_from_scikit_learn(gp)
        hyp_phase.append(hyperparameters)
        kinv_dot_y_phase.append(kinv_dot_y)
    
    f['hyp_phi'] = np.array(hyp_phase)
    f['kinv_dot_y_phi'] = np.array(kinv_dot_y_phase)
    
    print f['hyp_phi'][:].shape
    print f['kinv_dot_y_phi'][:].shape
    
    f.close()

In [19]:
filename = '../../data/TEOBv4_20hz/SEOBNRv4T_surrogate_v1.0.0.hdf5'
lalsuite_spline_surrogate_format(filename, sur)

(40,)
(39,)
Writing amplitude bases...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 (40, 7)
(40, 1359)

Writing phase bases...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 (39, 7)
(39, 1359)


## Testing

In [24]:
%%bash
#h5ls ../../data/TEOBv4_20hz/SEOBNRv4T_surrogate_v1.0.0.hdf5
#h5dump -n 1 ../../data/TEOBv4_20hz/SEOBNRv4T_surrogate_v1.0.0.hdf5
h5dump -a Description ../../data/TEOBv4_20hz/SEOBNRv4T_surrogate_v1.0.0.hdf5

HDF5 "../../data/TEOBv4_20hz/SEOBNRv4T_surrogate_v1.0.0.hdf5" {
ATTRIBUTE "Description" {
   DATATYPE  H5T_STRING {
      STRSIZE H5T_VARIABLE;
      STRPAD H5T_STR_NULLTERM;
      CSET H5T_CSET_ASCII;
      CTYPE H5T_C_S1;
   }
   DATASPACE  SCALAR
   DATA {
   (0): "
           ********************************************************************************
           Data for the SEOBNRv4T_surrogate waveform (aligned-spin BNS with tidal interactions).
           
           See B. Lackey, M. Puerrer, A. Taracchini. arXiv:xxxx.xxxx.
           
           Parameter ranges:
           * 1.0/3.0 <= q <= 1.0
           * -0.5 <= spin_1z <= 0.5
           * -0.5 <= spin_2z <= 0.5
           * 0.0 <= lambda_1 <= 5000.0
           * 0.0 <= lambda_2 <= 5000.0
           
           This surrogate was built using the SEOBv4T waveform.
           
           The first spline node for dphase is not listed since it is the same as the first
           node for damp, and dphase = 0 for the first 

# Compare TaylorF2 to lalsimulation version

In [12]:
# Note: these functions are not callable outside this source file, even though they are XLAL
# Therefore I reimplement them here.
# See LALSimInspiralPNCoefficients.c

def XLALSimInspiralTaylorF2Phasing_10PNTidalCoeff(mByM):
    return (-288. + 264.*mByM)*mByM*mByM*mByM*mByM;

def XLALSimInspiralTaylorF2Phasing_12PNTidalCoeff(mByM):
    return (-15895./28. + 4595./28.*mByM + 5715./14.*mByM*mByM - 325./7.*mByM*mByM*mByM)*mByM*mByM*mByM*mByM;

In [13]:
def pn_phase_lal(mfs, eta, chi1, chi2, lambda1, lambda2):
    """This is essentially the way LALSimulation calculates the TaylorF2 phase array.
    """
    d = np.sqrt(1.0-4.0*eta)
    X1 = 0.5*(1.0+d)
    X2 = 0.5*(1.0-d)
    
    # Fake masses
    mtot = 2.0
    m1 = X1*mtot
    m2 = X2*mtot
    
    # Specify PN order for spin terms
    extraParams = lal.CreateDict()
    lalsimulation.SimInspiralWaveformParamsInsertPNSpinOrder(
        extraParams, lalsimulation.SIM_INSPIRAL_SPIN_ORDER_35PN)
   
    # Calculate coefficients of each power of v
    pn = lalsimulation.SimInspiralTaylorF2AlignedPhasing(
        m1*lal.MSUN_SI, m2*lal.MSUN_SI, chi1, chi2, extraParams)
    #print pn.v
    
    # Manually add the tidal parameters
    
    pn.v[10] = pn.v[0]*(lambda1*XLALSimInspiralTaylorF2Phasing_10PNTidalCoeff(X1)
                        + lambda2*XLALSimInspiralTaylorF2Phasing_10PNTidalCoeff(X2))
    pn.v[12] = pn.v[0]*(lambda1*XLALSimInspiralTaylorF2Phasing_12PNTidalCoeff(X1)
                        + lambda2*XLALSimInspiralTaylorF2Phasing_12PNTidalCoeff(X2))
    
    #print pn.v[0]
    #print pn.v/pn.v[0]
    #print pn.vlogv/pn.v[0]/2
    #print pn.v[10]/pn.v[0], pn.v[12]/pn.v[0]
    
    phases = []
    for mf in mfs:
        v = (lal.PI * mf)**(1.0/3.0)
        logv = log(v)
    
        v2 = v * v;
        v3 = v * v2;
        v4 = v * v3;
        v5 = v * v4;
        v6 = v * v5;
        v7 = v * v6;
        v8 = v * v7;
        v9 = v * v8;
        v10 = v * v9;
        v12 = v2 * v10;
        phasing = 0.0;
        
        phasing += pn.v[7] * v7;
        phasing += (pn.v[6] + pn.vlogv[6] * logv) * v6;
        phasing += (pn.v[5] + pn.vlogv[5] * logv) * v5;
        phasing += pn.v[4] * v4;
        phasing += pn.v[3] * v3;
        phasing += pn.v[2] * v2;
        phasing += pn.v[1] * v;
        phasing += pn.v[0];
        
        # Tidal terms in phasing
        phasing += pn.v[12] * v12;
        phasing += pn.v[10] * v10;
        
        phasing /= v5;
        phases.append(phasing)
    
    return np.array(phases)

In [14]:
# Check that you reproduce the LALSimulation version to double precision

x = np.array([0.46, -0.7, 0.2, 5000, 500])
mf = np.logspace(-5, -1, 5)
tbymc = 0
phic = 0

q, chi1, chi2, lambda1, lambda2 = x
eta = q/(1.0+q)**2

lal_phase = pn_phase_lal(mf, eta, chi1, chi2, lambda1, lambda2)
ben_phase = -f2.taylorf2_phase(mf, tbymc, phic, eta, chi1, chi2, lambda1, lambda2)+np.pi/4

print eta, q, chi1, chi2, lambda1, lambda2
print mf

print lal_phase
print ben_phase
print lal_phase - ben_phase
print lal_phase/ben_phase - 1.0

0.215800337774 0.46 -0.7 0.2 5000.0 500.0
[  1.000000e-05   1.000000e-04   1.000000e-03   1.000000e-02   1.000000e-01]
[  3.486578e+06   7.528378e+04   1.372043e+03  -1.378574e+02  -4.084225e+03]
[  3.486578e+06   7.528378e+04   1.372043e+03  -1.378574e+02  -4.084225e+03]
[  1.862645e-09  -5.820766e-11  -1.591616e-12   2.842171e-14   9.094947e-13]
[  4.440892e-16  -7.771561e-16  -1.110223e-15  -2.220446e-16  -2.220446e-16]
